[PyHealth](https://pyhealth.readthedocs.io/en/latest/) is a Python library designed for healthcare data analysis, providing a
range of tools and functionalities that simplify the process of building, training, and
evaluating healthcare models.

- Here, we will use the PyHealth package to handle <u>readmission prediction</u> task on [MIMIC-IV-demo](https://physionet.org/content/mimic-iv-demo/2.2/) datasets. The package provides a whole suite of API
modules, covering MIMIC-IV data processing, DNN model initialization, training,
and evaluation. 

- The readmission task is defined by <u>using the procedure, medication, diagnosis codes from the current visit to predict
whether the patient will be readmitted into ICU within 7 days</u>.

Let us start step by step.

## STEP1: dataset processing

The first step is to preprocess the MIMIC-IV-demo data. Since we are unable to distribute the original MIMIC-IV dataset, we use the [MIMIC-IV-demo dataset](https://physionet.org/content/mimic-iv-demo/2.2/) as the substitute here.

The MIMIC-IV and other EHR
databases all have heterogeneous and complicated file structures. Each type of clinical
events will maintain a CSV table while these tables are connected by the unique patient
visit ID. 

Before feeding the dataset into deep learning models, researchers usually join
tables together and filter invalid records, etc. Here, we leverage
the diagnosis, procedure, and medication tables as feature entries for the readmission
prediction tasks. Lucikly, PyHealth provides the [pyhealth.datasets.MIMIC4Dataset](https://pyhealth.readthedocs.io/en/latest/api/datasets/pyhealth.datasets.MIMIC4Dataset.html) API to handle the data processing step.

Calling the MIMIC4Dataset API requires the root of MIMIC-IV-demo database and the
data tables that we want to use for features (diagnoses, procedures, and prescriptions).

In [2]:
from pyhealth.datasets import MIMIC4Dataset

mimic4_ds = MIMIC4Dataset(
    # if you have the access to MIMIC-IV dataset, please replace the following root
    root="https://storage.googleapis.com/pyhealth/mimiciv-demo/hosp",
    tables=["diagnoses_icd", "procedures_icd", "prescriptions"],
)

mimic4_ds.stat()

/home/chaoqiy2/miniconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
finish basic patient information parsing : 1.3815855979919434s
finish parsing diagnoses_icd : 1.0474581718444824s
finish parsing procedures_icd : 1.006455659866333s
finish parsing prescriptions : 1.826772928237915s


Mapping codes: 100%|███████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 5759.82it/s]


Statistics of base dataset (dev=False):
	- Dataset: MIMIC4Dataset
	- Number of patients: 100
	- Number of visits: 275
	- Number of visits per patient: 2.7500
	- Number of events per visit in diagnoses_icd: 16.3855
	- Number of events per visit in procedures_icd: 2.6255
	- Number of events per visit in prescriptions: 65.6945



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC4Dataset\n\t- Number of patients: 100\n\t- Number of visits: 275\n\t- Number of visits per patient: 2.7500\n\t- Number of events per visit in diagnoses_icd: 16.3855\n\t- Number of events per visit in procedures_icd: 2.6255\n\t- Number of events per visit in prescriptions: 65.6945\n'

In [3]:
# query the patient level information

patients = mimic4_ds.patients
patient_id, patient_obj = list(patients.items())[2]
patient_id, patient_obj

('10001725', Patient 10001725 with 1 visits)

In [4]:
# query the visit level information

visits = patient_obj.visits
visit_id, visit_obj = list(visits.items())[0] # the 0-th visit
visit_id, visit_obj

('25563031',
 Visit 25563031 from patient 10001725 with 91 events from tables ['diagnoses_icd', 'procedures_icd', 'prescriptions'])

In [5]:
# query the event level information

diagnoses = visit_obj.get_event_list("diagnoses_icd")
diagnosis = diagnoses[0] # the 0-th diagnosis
diagnosis

Event with ICD9CM code 78829 from table diagnoses_icd

In [6]:
# query the event level information

prescriptions = visit_obj.get_event_list("prescriptions")
prescription = prescriptions[0] # the 0-th prescription
prescription

Event with NDC code 68084008901 from table prescriptions

## STEP2: Task definition

To define the healthcare AI task, the [pyhealth.tasks](https://pyhealth.readthedocs.io/en/latest/api/tasks.html) module provides a different variety of example task functions. Each of them defines a unique healthcare
AI task. 

- Following the readmission task example (below), all task functions input
a pyhealth.data.Patient object and then output a list of samples containing the “X"
and “Y" for supervised learning. If the task is defined as a patient-level prediction task,
then the output is one sample; if it is a visit-level prediction task, then the output is a
list of samples, one for each visit.
Readmission prediction is a visit-level clinical predictive task. Given the above
structured dataset, we need to extract the diagnosis, procedure, and prescription infor-
mation as the features and compare the time gap between the current visit and next
visit to decide whether the patient will be readmitted into hospital within 7 days. If
the answer is Yes, then we set label to be 1; otherwise, the label is 0. It is a binary
classification task.
Luckily, 

PyHealth provides a default readmission prediction task with time window
as an argument (In this task, the threshold is 7 days).

In [7]:
def readmission_prediction_mimic4_fn(patient, time_window=7):
    """Processes a single patient for the readmission prediction task.
    ...
    """
    samples = []

    # we will drop the last visit since we cannot tell its label
    for i in range(len(patient) - 1):
        visit = patient[i]
        next_visit = patient[i + 1]

        # get time difference between current visit and next visit
        time_diff = (next_visit.encounter_time - visit.encounter_time).days
        readmission_label = 1 if time_diff < time_window else 0

        conditions = visit.get_code_list(table="diagnoses_icd")
        procedures = visit.get_code_list(table="procedures_icd")
        drugs = visit.get_code_list(table="prescriptions")
        # exclude: visits without condition, procedure, or drug code
        if len(conditions) * len(procedures) * len(drugs) == 0:
            continue
        samples.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                "conditions": conditions,
                "procedures": procedures,
                "drugs": drugs,
                "label": readmission_label,
            }
        )

        # use patient or visit level information for cohort selection
        # ...
    return samples

In [8]:
# STEP 2: define the readmission prediction task
readmission_dataset = mimic4_ds.set_task(
    lambda x: readmission_prediction_mimic4_fn(x, time_window=7)
)
readmission_dataset.stat()

Generating samples for <lambda>: 100%|████████████████████████████████████████████| 100/100 [00:00<00:00, 14952.42it/s]

Statistics of sample dataset:
	- Dataset: MIMIC4Dataset
	- Task: <lambda>
	- Number of samples: 104
	- Number of patients: 42
	- Number of visits: 104
	- Number of visits per patient: 2.4762
	- conditions:
		- Number of conditions per sample: 17.6827
		- Number of unique conditions: 843
		- Distribution of conditions (Top-10): [('4019', 26), ('E039', 21), ('E785', 20), ('2724', 18), ('Z87891', 18), ('V1582', 15), ('F329', 15), ('Z794', 15), ('N179', 14), ('2859', 12)]
	- procedures:
		- Number of procedures per sample: 3.3077
		- Number of unique procedures: 199
		- Distribution of procedures (Top-10): [('3897', 15), ('02HV33Z', 14), ('966', 10), ('5491', 7), ('9671', 7), ('3893', 7), ('5A1D70Z', 7), ('5A1945Z', 6), ('3E0G76Z', 6), ('9604', 5)]
	- drugs:
		- Number of drugs per sample: 38.2500
		- Number of unique drugs: 928
		- Distribution of drugs (Top-10): [('0', 104), ('63323026201', 74), ('00409490234', 56), ('00338004904', 53), ('00904224461', 51), ('00904516561', 49), ('3839605

"Statistics of sample dataset:\n\t- Dataset: MIMIC4Dataset\n\t- Task: <lambda>\n\t- Number of samples: 104\n\t- Number of patients: 42\n\t- Number of visits: 104\n\t- Number of visits per patient: 2.4762\n\t- conditions:\n\t\t- Number of conditions per sample: 17.6827\n\t\t- Number of unique conditions: 843\n\t\t- Distribution of conditions (Top-10): [('4019', 26), ('E039', 21), ('E785', 20), ('2724', 18), ('Z87891', 18), ('V1582', 15), ('F329', 15), ('Z794', 15), ('N179', 14), ('2859', 12)]\n\t- procedures:\n\t\t- Number of procedures per sample: 3.3077\n\t\t- Number of unique procedures: 199\n\t\t- Distribution of procedures (Top-10): [('3897', 15), ('02HV33Z', 14), ('966', 10), ('5491', 7), ('9671', 7), ('3893', 7), ('5A1D70Z', 7), ('5A1945Z', 6), ('3E0G76Z', 6), ('9604', 5)]\n\t- drugs:\n\t\t- Number of drugs per sample: 38.2500\n\t\t- Number of unique drugs: 928\n\t\t- Distribution of drugs (Top-10): [('0', 104), ('63323026201', 74), ('00409490234', 56), ('00338004904', 53), ('009

In [9]:
# look at the first sample

readmission_dataset[0]

{'visit_id': '22595853',
 'patient_id': '10000032',
 'conditions': ['5723',
  '78959',
  '5715',
  '07070',
  '496',
  '29680',
  '30981',
  'V1582'],
 'procedures': ['5491'],
 'drugs': ['63323026201',
  '19515089452',
  '0',
  '00245004101',
  '51079007220',
  '63739054410',
  '00904198861',
  '00135019502',
  '00006022761',
  '61958070101',
  '00173068224',
  '00487980125',
  '51079007320'],
 'label': 0}

In [16]:
from pyhealth.datasets import split_by_patient, get_dataloader

# split the dataset into train/val/test
train_dataset, val_dataset, test_dataset = split_by_patient(
    readmission_dataset, [0.6, 0.2, 0.2]
)
train_dataloader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

## STEP3: initialize ML models

The [pyhealth.models](https://pyhealth.readthedocs.io/en/latest/api/models.html) module provides a large variety of ML models for users to
choose from. These methods can be broadly categorized into: (i) general deep learning
models and (ii) healthcare specific deep learning models. For healthcare specific DL
models, we are constantly implementing new ones and add them into our package.

In this section, we will use the simple three-layer multi-layer perceptron (MLP) in
PyHealth, specifying the number of layers, the number of hidden neurons in each layer,
and the activation functions. More advanced models and examples can be found in
later sections or visit the [PyHealth website](https://pyhealth.readthedocs.io/en/latest/).

In model initialization, we input the keys listed in the sample dictionary. To leverage
three types of clinical event information, we use “conditions", “procedures", “drugs"
as keywords for features, and we input the “label" as the readmission prediction label.

In [21]:
from pyhealth.models import MLP

# STEP 3: define model
model = MLP(
    dataset=readmission_dataset,
    feature_keys=["conditions", "procedures", "drugs"],
    label_key="label",
    mode="binary",
    embedding_dim=16,
    hidden_dim=16,
    n_layers=2,
    activation="relu",
)

## STEP 4: model training

In PyHealth, the [pyhealth.trainer.Trainer](https://pyhealth.readthedocs.io/en/latest/api/trainer.html) module serves as a powerful tool for
managing the training process of all models. 

This trainer simplifies logging during
training, automatically selects available devices, and conveniently saves the best model
to disk. The trainer module offers a range of selectable arguments for customization.

Let’s proceed with the programming steps outlined below.

In [22]:
from pyhealth.trainer import Trainer

# STEP 4: define trainer
trainer = Trainer(model=model)

trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=50,
    monitor="roc_auc",
) # model is training ...


MLP(
  (embeddings): ModuleDict(
    (conditions): Embedding(845, 16, padding_idx=0)
    (procedures): Embedding(201, 16, padding_idx=0)
    (drugs): Embedding(930, 16, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (activation): ReLU()
  (mlp): ModuleDict(
    (conditions): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=16, bias=True)
    )
    (procedures): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=16, bias=True)
    )
    (drugs): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=16, bias=True)
    )
  )
  (fc): Linear(in_features=48, out_features=1, bias=True)
)
Metrics: None
Device: cuda

Training:
Batch size: 32
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.001}
Weight decay: 0

Epoch 0 / 50:   0%|          | 0/2 [00:00<?, ?it/s]

--- Train epoch-0, step-2 ---
loss: 0.6956


Evaluation: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 217.30it/s]

--- Eval epoch-0, step-2 ---
pr_auc: 0.5418
roc_auc: 0.5714
f1: 0.5333
loss: 0.6931
New best roc_auc score (0.5714) at epoch-0, step-2



...

--- Eval epoch-48, step-98 ---
pr_auc: 0.4999
roc_auc: 0.5397
f1: 0.5000
loss: 0.6736



Epoch 49 / 50:   0%|          | 0/2 [00:00<?, ?it/s]

--- Train epoch-49, step-100 ---
loss: 0.4221


Evaluation: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 233.82it/s]

--- Eval epoch-49, step-100 ---
pr_auc: 0.4999
roc_auc: 0.5397
f1: 0.5000
loss: 0.6751
Loaded best model


## STEP5: model evaluation

Following model training, we offer two evaluation methods for the optimized deep
learning model through the [pyhealth.metrics](https://pyhealth.readthedocs.io/en/latest/api/metrics.html). 

PyHealth offers an array
of metrics suitable for assessing binary classification, multi-class classification, and
multi-label classification problems, alongside fairness metrics and advanced model
calibration metrics.

In [24]:
# method 1
result = trainer.evaluate(test_dataloader)
print (result)

# method 2
from pyhealth.metrics.binary import binary_metrics_fn

y_true, y_prob, loss = trainer.inference(test_dataloader)
binary_metrics_fn(
    y_true,
    y_prob,
    metrics=["pr_auc", "roc_auc", "f1"]
)

Evaluation: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 205.15it/s]


{'pr_auc': 0.5818931986585775, 'roc_auc': 0.39772727272727276, 'f1': 0.0, 'loss': 0.7247440218925476}


Evaluation: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 155.38it/s]


{'pr_auc': 0.5818931986585775, 'roc_auc': 0.39772727272727276, 'f1': 0.0}

- Note that the performance is bad on MIMIC-IV-demo dataset since it only contains 100 patients, which makes the model overfit. You could try to download the original MIMIC-IV dataset and re-run the pipeline.